In [1]:
from os import listdir, getcwd
import cv2 as cv
import numpy as np
import time

In [15]:
def get7Moments(img):
    '''Calcular los momentos de Hu de la imagen'''

    img = cv.bitwise_not(img)
    hu = cv.HuMoments(cv.moments(img))

    for i in range(0, 7):
        hu[i] = -1 * np.sign(hu[i]) * np.log10(np.abs(hu[i]))
    
    hu = hu.reshape((1, 7)).tolist()[0]
    
    return hu

In [89]:
def processImages(HUs):
    '''Recorrer las imagenes, calcular sus momentos y agregar los promedios al diccionario'''

    CWD = './'

    DIGITS = [0,1,2,3,4,5,6,7,8,9]

    # Recorrer los folders de los digitos
    for number in DIGITS:

        # Cronometro
        startTime = time.perf_counter()

        folder = CWD + str(number) + '/'

        # Ver los archivos del folder
        files = listdir(folder)
        
        # Contenedores de los momentos de Hu de las imagenes del folder
        momento1 = []
        momento2 = []
        momento3 = []
        momento4 = []
        momento5 = []
        momento6 = []
        momento7 = []

        # Recorremos los archivos del folder
        for file in files:
            
            # Leer imagen
            img = cv.imread(folder + file,0)

            # Calcular los momentos de la imagen
            #momentos = get7Moments(img)
            momentos = cv.moments(img)
            momentos = cv.HuMoments(momentos)
            # Agregar cada momento a cada lista para poder calcular sus promedios al final
            momento1 += momentos[0].tolist() 
            momento2 +=  momentos[1].tolist() 
            momento3 += momentos[2].tolist() 
            momento4 += momentos[3].tolist() 
            momento5 +=  momentos[4].tolist() 
            momento6 +=  momentos[5].tolist() 
            momento7 +=  momentos[6].tolist() 

        # Calcular el promedio de los momentos de Hu para este digito
        momentosHuDigito = [

            sum(momento1)/len(momento1),
            sum(momento2)/len(momento2),
            sum(momento3)/len(momento3),
            sum(momento4)/len(momento4),
            sum(momento5)/len(momento5),
            sum(momento6)/len(momento6),
            sum(momento7)/len(momento7)

        ]

        # Agregar promedio de momentos al diccionario
        HUs[number] = momentosHuDigito

        print("Procesado de " + str(number) + ': ' + str(time.perf_counter() - startTime) + " s (" + str(len(files)) + ")")

In [90]:
HUs = {}

processImages(HUs)

Procesado de 0: 0.05968360000042594 s (324)
Procesado de 1: 0.04885230000036245 s (280)
Procesado de 2: 0.06169080000017857 s (335)
Procesado de 3: 0.05301869999948394 s (285)
Procesado de 4: 0.05202259999987291 s (268)
Procesado de 5: 0.06375789999947301 s (319)
Procesado de 6: 0.058065000000169675 s (302)
Procesado de 7: 0.04199430000062421 s (225)
Procesado de 8: 0.06838609999977052 s (327)
Procesado de 9: 0.05235189999984868 s (283)


In [91]:
for key in HUs.keys():

    print(HUs[key])


[0.0015516787098529005, 1.7674411198111234e-06, 3.4447932828946614e-11, 2.8462507182739372e-11, 1.4240867862307583e-20, 6.46019038253848e-14, 1.6834390245775724e-21]
[0.0037842797518092947, 5.529041357934756e-05, 3.4583516953030024e-09, 2.6669459533043724e-09, 1.1133391788191514e-16, 1.0201948923505193e-11, -1.0297784342947463e-17]
[0.0013592908189216176, 1.0623282088634982e-06, 6.12590233529266e-11, 3.9745634431154096e-11, 1.2071354431835816e-20, 4.4068599361388346e-14, 2.9547323385862314e-22]
[0.001416729611118886, 1.2113490609892818e-06, 4.052762672659612e-11, 1.4284604775367497e-11, 4.1197128483839435e-22, -1.472619441167016e-15, -7.67744814652639e-22]
[0.0015997814473864475, 1.8480624029314437e-06, 1.1029725476024232e-10, 9.180539902380717e-11, 2.7119531703124505e-20, 1.4893582788977585e-13, -2.2749606974145355e-21]
[0.0014151090609298977, 1.2351065945862248e-06, 2.810717461448768e-11, 1.9104188213157123e-11, 4.273423201576242e-21, 2.2938813740030517e-14, 2.0979528241784669e-22]
[

In [114]:
test = cv.imread('./7.jpg',0)
momentos = cv.moments(test)
momentos = cv.HuMoments(momentos).flatten()

dif = []

momentos = np.array(momentos)

for key in HUs.keys():
    
    dif += [sum(abs(momentos - HUs[key]))]
    
pred = min(dif)

print(dif.index(pred))

5
